# Neural network with one Hidden Layer

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from planar_utils import plot_decision_boundary, sigmoid, load_planar_dataset, load_extra_datasets

In [3]:
X, Y = load_planar_dataset()

# Defining the neural network Structure

In [4]:
def layer_sizes(X, Y):
    n_x = X.shape[0] # size of input layer
    n_h = 4          # size of hidden layer
    n_y = Y.shape[0] # size of output layer
    return (n_x, n_h, n_y)

# Initialize the model parameters

In [5]:
def initialize_parameters(n_x, n_h, n_y):
    
    np.random.seed(2) 
    
   
    W1 = np.random.randn(n_h,n_x)* 0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)* 0.01
    b2 =  np.zeros((n_y,1))
   
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

# Forward propagation 

In [6]:
def forward_propagation(X,parameters):
    
    W1 = parameters["W1"]  # (4,2)
    b1 = parameters["b1"]  # (4,1)
    W2 = parameters["W2"]  # (1,4)
    b2 = parameters["b2"]  # (1,1)
   
    
    Z1 = np.dot(W1,X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1) + b2
    A2 = sigmoid(Z2)
    
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

# Cost

In [7]:
def compute_cost(A2, Y, parameters):
 
    m = Y.shape[1]                                                               # number of example
    
    
    logprobs = np.multiply(Y ,np.log(A2)) + np.multiply((1-Y), np.log(1-A2))
    cost = (-1/m) * np.sum(logprobs) 

    cost = np.squeeze(cost)     
    cost = cost.astype(float)
    assert(isinstance(cost, float))
    
    return cost

# Backward propagation

In [8]:
def backward_propagation(parameters, cache, X, Y):
    
    m = X.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    A1 = cache["A1"]
    A2 = cache['A2']
    
    
    dZ2 = A2-Y
    dW2 = 1/m* np.dot(dZ2,A1.T)
    #print("dW2=" + str(dW2.shape))
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T,dZ2) * (1 - np.power(A1,2)) 
    #print("dz1="+ str(dZ1.shape))
    dW1 = (1/m) * np.dot(dZ1,X.T)
    #print("dW1="+ str(dW1.shape))
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    #print("db1="+ str(db1.shape))
    
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

# Update Parameters

In [9]:
def update_parameters(parameters, grads, learning_rate = 1.2):
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
   
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

# Model

In [10]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):
  
    np.random.seed(3)
    
    # calling layer_sizes
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # initializing with random and zeros
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    

    for i in range(0, num_iterations):
         
       
        # Forward propagation
        A2, cache = forward_propagation(X, parameters)
        
        # Cost function. Inputs 
        cost = compute_cost(A2, Y, parameters)
 
        # Backpropagation. Inputs
        grads = backward_propagation(parameters, cache, X, Y)
 
        # Gradient descent  update
        parameters = update_parameters(parameters, grads)
        
   
        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [11]:
parameters = nn_model(X, Y, n_h = 4, num_iterations = 10000, print_cost=True)

Cost after iteration 0: 0.693048
Cost after iteration 1000: 0.288083
Cost after iteration 2000: 0.254385
Cost after iteration 3000: 0.233864
Cost after iteration 4000: 0.226792
Cost after iteration 5000: 0.222644
Cost after iteration 6000: 0.219731
Cost after iteration 7000: 0.217504
Cost after iteration 8000: 0.219504
Cost after iteration 9000: 0.218571


# Prediction and Accuracy

In [12]:
def predict(parameters, X):
    A2, cache = forward_propagation(X, parameters)
    predictions = (A2 > 0.5)
    return predictions

In [13]:
predictions = predict(parameters, X)
print ('Accuracy: %d' % float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100) + '%')

Accuracy: 90%
